In [ ]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
import folium
import numpy as np
import os
import pandas as pd

In [ ]:
def visualize(filename, data_gps):
    
    data_pred = pd.read_csv(filename)
    print(np.unique(data_pred['anomalies_category'].values, return_counts=True))
    def concat_with_event(row):
        data = data_pred[((data_pred["time"] >= row['time'] - 10) & (data_pred["time"] <= row['time'] + 10))].reset_index()
        try:
            return data['anomalies_category'].values[0]
        except:
            return np.nan
    
    data_gps['anomalies_category'] = data_gps.apply(concat_with_event, axis=1)
    data_gps = data_gps.dropna()
    data_gps['speed'] = data_gps['speed'] * 3.6
    data_gps['speed'] = data_gps['speed'].apply(round)
    coords_critical = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='critical'].values.tolist()
    coords_negligible = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='negligible'].values.tolist()
    coords_significant = data_gps[['lat', 'lng', "speed"]][data_gps['anomalies_category']=='significant'].values.tolist()
    mapit = folium.Map(location=[data_gps['lat'].min(), data_gps['lng'].min()], zoom_start=15)

    for coord in coords_negligible:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: green;">{coord[2]}</div>""")).add_to(mapit)

    for coord in coords_significant:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: black;">{coord[2]}</div>""")).add_to(mapit)

    for coord in coords_critical:
        folium.Marker(location=[coord[0], coord[1]], icon = folium.DivIcon(html=f"""<div style="color: red;">{coord[2]}</div>""")).add_to(mapit)
    
    return mapit

In [ ]:
data_gps = pd.read_csv(os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', 'gps.csv'))
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer.csv'
visualize(os.path.join('data', 'test', 'predict', 'linear', filename), data_gps)

In [ ]:
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope.csv'
visualize(os.path.join('data', 'test', 'predict', 'linear', filename), data_gps)

In [ ]:
data_gps = pd.read_csv(os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', 'gps.csv'))
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer.csv'
visualize(os.path.join('data', 'test', 'predict', 'non-linear', filename), data_gps)

In [ ]:
data_gps = pd.read_csv(os.path.join('data', 'test', 'sensorlab_2020-02-15-06.52.25', 'gps.csv'))
filename = 'sensorlab_2020-02-15-06.52.25_accelerometer_gyroscope.csv'
visualize(os.path.join('data', 'test', 'predict', 'non-linear', filename), data_gps)